In [1]:
from koebe.datastructures.dcel import *

class Tiling(DCEL):
    
    def __init__(self, outerFaceData = None):
        super().__init__(outerFaceData)

        self.dartLevels = []
        self.edgeLevels = []
        self.faceLevels = []
        self.subdivisionLevel = 1
        
        self.Vertex = TilingVertex
        self.Dart   = TilingDart
        self.Face   = Tile

    def addLevel(self):
        old_darts, old_edges, old_faces = self.darts, self.edges, self.faces
        
        self.dartLevels.append(self.darts)
        self.edgeLevels.append(self.edges)
        self.faceLevels.append(self.faces)
        
        self.darts = []
        self.edges = []
        self.faces = []
        
        self.subdivisionLevel += 1
        self.outerFace = self.Face(dcel = self)
        
        return old_darts, old_edges, old_faces

class TilingVertex(Vertex):
    def __init__(self, dcel, data = None):
        super().__init__(dcel = dcel, data = data)
        self.level = dcel.subdivisionLevel
    
class TilingDart(Dart):
    def __init__(self, 
                 dcel, 
                 edge   = None, 
                 origin = None, 
                 face   = None,
                 prev   = None,
                 next   = None,
                 twin   = None, 
                 data   = None):
        super().__init__(dcel = dcel, edge = edge, origin = origin, face = face, 
                         prev = prev, next = next, twin = twin, data = data)
        self.subdarts = []

class Tile(Face):
    def __init__(self, dcel, tileType=None, data=None):
        super().__init__(dcel = dcel, data = data)
        self.tileType = tileType
        self.subtiles = []
        
class TilingRules:
    
    def __init__(self):
        self.prototiles = {}
    
    def createPrototile(self, tileType, tileVerts):
        self.prototiles[tileType] = Prototile(self, tileType, tileVerts)
        return self.prototiles[tileType]
    
    def getPrototile(self, tileType):
        return self.prototiles[tileType]
    
    def createInitialTile(self, tileType):
        prototile = self.getPrototile(tileType)
        tiling    = Tiling.generateCycle(vdata = prototile.tileVerts)
        tiling.faces[-1].tileType = tileType
        return tiling
    
    def generateTiling(self, initialTileType, depth = 1):
        tiling = self.createInitialTile(initialTileType)
        for _ in range(depth): tilingPass(tiling, self)
        return tiling

    
    
class Prototile:
    
    def __init__(self, tilingRules: TilingRules, tileType: str, tileVerts: str):
        """
        Args: 
            tilingRules: The TilingRules object that this prototile is a part of. 
            tileVerts: The name of the vertices. For example, "ABCDEF" is a 
                six sided tile with vertex names A, B, C, D, E, and F. 
        """
        self.tilingRules  = tilingRules
        self.tileType     = tileType
        self.tileVerts    = tileVerts
        self.splitRules   = dict()
        self.splitFn      = dict()
        self.childDartsOf = dict()
        self.parentDartOf = dict()
        self.newVertRules = list()
        self.newVertFn    = list()
        self.subtiles     = list()
    
    def addSplitEdgeRule(self, edge, newverts, fn = None):
        """
        Adds a rule to split an edge by introducing a series of new verts. For example
        if edge is "AB" and newverts is "a" this command will split the edge AB into
        Aa and aB. If newverts is "abc" then AB is split into Aa ab bc and cB. 
        Args:
            self
            edge: The name of the edge to split, like "AB"
            newverts: The new vertices to introduce, like "a". 
            fn (Optional): (Dart, List[Vertex])->None
        """
        if edge[0] in self.splitRules:
            raise PrototileFormationError("There appear to be two splits beginning with the same vertex. This probably means you either attempted to split the same edge twice, or did not orient your edges consistently.")
        self.splitRules[edge[0]] = (edge, newverts)
        if fn != None:
            self.splitFn[edge[0]] = fn
        splitEdge = [edge[0]] + list(newverts) + [edge[1]]
        self.parentDartOf.update(
            [((splitEdge[i], splitEdge[i+1]), edge) 
             for i in range(len(splitEdge) - 1)]
        )
        self.childDartsOf[edge] = [(splitEdge[i], splitEdge[i+1])
                                   for i in range(len(splitEdge) - 1)]
    
    def addSplitEdgeRules(self, splitCommands):
        """
        Args: 
            self
            splitCommands: A list of tuples of the form (edge, newverts) that directs
                the tiling to split this edge by introducing a series of new vertices. 
                For example, (("A", "B"), ("a")) would split the edge AB into two edges Aa and 
                aB. 
                
                The tuples may optionally contain a third parameter, a function fn of
                type (Face, (Vertex, Vertex), List[Vertex]) -> None that sets any additional
                data when the split occurs.
        """
        for command in splitCommands: 
            self.addSplitEdgeRule(*command)
    
    def addNewVertexRule(self, vertName, fn = None):
        """
        Args:
            vertName: the name of the new vertex
            fn: (Face) -> None
        """
        self.newVertRules.append(vertName)
        if fn != None:
            self.newVertFn[vertName] = fn
    
    def addNewVertexRules(self, vertNames):
        for vertName in vertNames:
            self.addNewVertexRule(vertName)
    
    def addSubtile(self, subtileType, subtileVerts):
        self.subtiles.append((subtileType, subtileVerts))
    
class PrototileFormationError(Exception):
    pass

def originNameToDartDict(tilingRules, tile):
    
    prototile = tilingRules.getPrototile(tile.tileType)
    theDarts  = tile.darts()
    
    if len(theDarts) != len(prototile.tileVerts):
        raise PrototileFormationError(
                f"Prototile vertex count ({len(theDarts)}) does not match tile"
                + f" vertex count for tile type {tile.tileType}."
        )
    
    return dict([(prototile.tileVerts[i], theDarts[i]) for i in range(len(theDarts))])

def subdivideTile(tilingRules, tiling, tile):
    prototile = tilingRules.getPrototile(tile)
    tile.subtiles = [Tile(tiling, prototile.subtiles[i][0]) for i in range(len(prototile.subtiles))]
    dartFrom = originNameToDartDict(tilingRules, tile)

def tilingPass(tilingRules, tiling):
    
    # tiles that need to be subdivided
    tiles = [tile for tile in tiling.faces if tile != tiling.outerFace]
    
    tiling.addLevel()
    
    dartsByNameForTile = [originNameToDartDict(tilingRules, tile) for tile in tiles]

def _old_setTileAnnotations(tilingRules, tiles):
    """
    For each tile adds a .vertexByName dictionary for mapping 
    """
    for tile in tiles: 
        prototile = tilingRules.getPrototile(tile.tileType)
        theDarts  = tile.darts()
        
        if len(theDarts) != len(prototile.tileVerts):
            raise PrototileFormationError(
                    f"Prototile vertex count ({len(theDarts)}) does not match tile"
                    + f" vertex count for tile type {tile.tileType}."
            )
        
        tile.vertexByName = {}
        # Build a mapping from vertex names to darts originating at the vertex
        # and vice versa. 
#         tile.vertexToDart   = {}
        for i in range(len(theDarts)):
            vertexName = prototile.tileVerts[i]
#             tile.vertexToDart[vertexName] = theDarts[i]
            theDarts[i].originName = vertexName
        
        tile.edgeNameToDart = {}
        for dart in theDarts:
            name = (dart.originName, dart.next.originName)
            tile.edgeNameToDart[name] = dart
            dart.edgeName = name
        
        tile.splitEdgeNameToDart = {}
    
def _old_tilingPass(tilingRules, tiling):
    
    # tiles that need to be subdivided
    tiles = [tile for tile in tiling.faces if tile != tiling.outerFace]
    
    # Annotate each tile with dictionaries to switch between 
    # vertex names and darts within the tile and edge names and darts. 
    setTileAnnotations(tilingRules, tiles)
            
    # For each tile, annotate any edge that needs to be split with its split rules. 
    for tile in tiles:
        prototile = tilingRules.getPrototile(tile.tileType)
        theDarts  = tile.darts()
        for dart in theDarts:
            if dart.edgeName in prototile.splitRules:
                dart.splitRule = prototile.splitRules[dart.edgeName]
            else:
                dart.splitRule = ""
    for dart in tiling.outerFace.darts():
        dart.splitRule = ""
    
    def annotateAndCheckEdge(edge):
        """
        Annotates the edge with .newVertexCount, the number of new vertices to subdivide it into. 
        Return:
            True iff. the split count is consistent on both sides of the edge. 
        """
        dart1, dart2 = edge.darts()
        if dart1.face == edge.dcel.outerFace:
            edge.newVertexCount = len(dart2.splitRule)
        elif dart2.face == edge.dcel.outerFace:
            edge.newVertexCount = len(dart1.splitRule)
        elif len(dart1.splitRule) == len(dart2.splitRule):
            edge.newVertexCount = len(dart2.splitRule)
        else:
            edge.newVertexCount = None
        return edge.newVertexCount != None
    
    def raiseEdgeError(edge):
        dart1, dart2 = edge.darts()
        tileName1 = dart1.face.tileType
        tileName2 = dart2.face.tileType
        splitCount1 = len(dart1.splitRule) + 1
        splitCount2 = len(dart2.splitRule) + 1
        
        raise PrototileFormationError(
                "Edge subdivision cannot be performed because "
                + f"edge {dart1.edgeName} of tile {tileName1}"
                + f" has to split into {splitCount1} edges"
                + " but is matched with "
                + f"edge {dart2.edgeName} of tile {tileName2}," 
                + f" which has to split into {splitCount2} edges"
        )
    
    # Check that each edge compiles:
    for edge in tiling.edges:
        if not annotateAndCheckEdge(edge):
            raiseEdgeError(edge)
    
    # Now we need to actually split each edge introducing new vertices
    # then add the new edges and stitch the tiles up. 

    def getSplitVertexNames(dart):
        """ For each dart get the vertex names for the split. For example
            if we're splitting a dart representing AB in a tiling by introducing
            a single vertex named a, then getSplitVertexNames will return ("A", "a", "B")
            as a tuple.
            
            Args: 
            dart: The dart that needs to be split. 
            
            Returns:
            A tuple containing the vertex names for the split version of the dart.
        """
        if dart.face != dart.dcel.outerFace:
            split_vertex_names = (dart.edgeName[0]) + dart.splitRule + (dart.edgeName[-1])
#             print(f"dart splitRule: {dart.edgeName} to {dart.splitRule} to form {split_vertex_names}")
        else:
            split_vertex_names = None
        return split_vertex_names
    
    def setVertexAndEdgeNames(dartList, vertexNames, parentTile, lastVertex):
        if vertexNames != None:
            for i in range(len(dartList)):
                d = dartList[i]
                d.edgeName = (vertexNames[i], vertexNames[i+1])
                if parentTile != d.dcel.outerFace:
                    parentTile.vertexByName[vertexNames[i]] = d.origin 
                    parentTile.splitEdgeNameToDart[d.edgeName] = d
            if parentTile != d.dcel.outerFace:
                parentTile.vertexByName[vertexNames[-1]] = lastVertex
    
    old_darts, old_edges, old_faces = tiling.addLevel() # Push all the current level info onto the various stacks and reset for new one. 
    
    # Split all edges/darts according to their split rules and store a mapping from new split dart names
    # to Dart objects in the corresponding tile. 
    for e in old_edges:
        
        # For each dart get the vertex names for the split. For example
        # if we're splitting a dart representing AB in a tiling by introducing
        # a single vertex named a, then getSplitVertexNames will return ("A", "a", "B")
        # as a tuple. 
        vertNames1 = getSplitVertexNames(e.aDart)
        vertNames2 = getSplitVertexNames(e.aDart.twin)
        
        # Create the new vertices along the edge and the new darts. 
        newVerts = [Vertex(e.dcel) for _ in range(e.newVertexCount)]  
        
        nDarts1 = ([Dart(e.dcel, origin = e.aDart.origin)] 
                   + [Dart(e.dcel, origin = v) for v in newVerts])
        
        nDarts2 = ([Dart(e.dcel, origin = e.aDart.dest)] 
                   + [Dart(e.dcel, origin = v) for v in reversed(newVerts)])
        
        setVertexAndEdgeNames(nDarts1, vertNames1, e.aDart.face, e.aDart.dest)
        setVertexAndEdgeNames(nDarts2, vertNames2, e.aDart.twin.face, e.aDart.origin)
        
        e.aDart.childDarts = nDarts1
        e.aDart.twin.childDarts = nDarts2
        
        e.childEdges = []
        for i in range(len(nDarts1)):
            nDarts1[i].makeTwin(nDarts2[len(nDarts2) - 1 - i])
            newEdge = Edge(e.dcel, aDart = nDarts1[i])
            e.childEdges.append(newEdge)
            nDarts1[i].edge = newEdge
            nDarts2[i].edge = newEdge
    
    # Split all tiles
    for tile in tiles:
        # Create any new vertices not obtained via splitting
        # and add them to the vertexByName map. 
        
        # Create a new face for each subtile getting either a dart
        # created in the edge split pass or creating a brand new dart
        
        # Loop over all the darts and find any that do not have a twin
        # for any that do not have a twin, set their twin correctly and
        # create an Edge between them
        
        pass
    print(tile.vertexByName)


def starTriangulateAllFaces(tiling):
    for face in tuple(tiling.faces):
        if face != tiling.outerFace:
            face.starTriangulate()

def tilingPass(tiling, tilingRules):
    
    #####
    # Subdivide a tile: 
    ##### 

    def subdivideTile(tiling, tile, tilingRules):

        prototile = tilingRules.getPrototile(tile.tileType) # Grab this tile's subdivision rules

        # Keep track of a few of the original items before we start the subdivision procedure
        originalDarts = tile.darts()

        originalDartFrom  = originNameToDartDict(tilingRules, tile) 
        vertNamed = dict([(vName, originalDartFrom[vName].origin) for vName in prototile.tileVerts])

        #####
        # 1. Create the new vertices. 
        #####

        # Create the new vertices required by each dart split:
        for i in range(len(originalDarts)):
            dart = originalDarts[i]
            splitRule = prototile.splitRules[prototile.tileVerts[i]] if prototile.tileVerts[i] in prototile.splitRules else (prototile.tileVerts[i], ())
            
            if dart.splitVertices == []:
                for vName in splitRule[1]:
                    vert = TilingVertex(tiling)
                    vertNamed[vName] = vert
                    dart.splitVertices.append(vert)
                dart.twin.splitVertices = list(reversed(dart.splitVertices))
            else:
                for vIdx in range(len(splitRule[1])):
                    vert = dart.splitVertices[vIdx]
                    vName = splitRule[1][vIdx]
                    vertNamed[vName] = vert

        # Create the other new vertices
        for vName in prototile.newVertRules:
            vertNamed[vName] = TilingVertex(tiling)

        #####
        # 2. Create the new darts and tiles.  
        #####

        dartNamed = {}
        nameOfDart = {}
        newDarts = []

        # Each tile is now split into subtiles and darts are created to surround each tile: 
        for subtileType, subtileVertNames in prototile.subtiles:
            subtile = Tile(tiling, subtileType)
            subtile.name = subtileVertNames
            tile.subtiles.append(subtile)
            subtile.parent = tile
            subtileDarts = [tiling.Dart(dcel = tiling, origin = vertNamed[vName], face=subtile) 
                            for vName in subtileVertNames]
            
            for i in range(len(subtileVertNames)):
                subtileDarts[i].name = subtileVertNames[i]
                subtileDarts[i].parent = None

            newDarts += subtileDarts
            subtile.aDart = subtileDarts[0]
            for i in range(len(subtileDarts)):
                subtileDarts[i-1].makeNext(subtileDarts[i]) # Stitches up the darts to be formed correctly.

            namesToDarts = [((subtileVertNames[i-1], subtileVertNames[i]), subtileDarts[i-1])
                             for i in range(len(subtileDarts))]
            
            dartNamed.update(namesToDarts)
            nameOfDart.update([(dart, name) for name, dart in namesToDarts])

        # Set the twin pointers for the internally created darts. 
        for i in range(len(newDarts)):
            origin, dest = nameOfDart[newDarts[i]]
            dart1 = dartNamed[(origin, dest)]
            if (dest, origin) in dartNamed and dart1.edge == None:
                dart2 = dartNamed[(dest, origin)]
                dart1.makeTwin(dart2)
                edge = tiling.Edge(tiling, dart1)
                dart1.edge = dart2.edge = edge

        # For each original dart, set its children list: 
        nameOfOriginalDart = dict([(originalDarts[i-1], (prototile.tileVerts[i-1], prototile.tileVerts[i])) 
                              for i in range(len(originalDarts))])

        for originalDart in originalDarts:
            name = nameOfOriginalDart[originalDart]
            originalDart.subdarts = [dartNamed[childName] 
                                     for childName in (prototile.childDartsOf[name] 
                                                       if name in prototile.childDartsOf 
                                                       else [name])]
            for subdart in originalDart.subdarts:
                subdart.parent = originalDart
    #####
    # Stitch the subtiles together. 
    #####
    def stitchSubTilesTogether(tiling):

        oldOuterFace = tiling.faceLevels[-1][0]
        oldFaces = tiling.faceLevels[-1]
        oldEdges = tiling.edgeLevels[-1]
        oldDarts = tiling.dartLevels[-1]

        for e in oldEdges:
            if e.aDart.face is not oldOuterFace and e.aDart.twin.face is not oldOuterFace:
                darts1 = e.aDart.subdarts
                darts2 = list(reversed(e.aDart.twin.subdarts))
                if len(darts1) != len(darts2):
                    raise PrototileFormationError(f"Edge ({e.aDart.name},{e.aDart.next.name}) of a {e.aDart.face.tileType} prototile is matched with edge ({e.aDart.twin.name},{e.aDart.twin.next.name}) of a {e.aDart.twin.face.tileType} prototile at level {e.dcel.subdivisionLevel-1}, but these do not split consistently.")
                for i in range(len(darts1)):
                    dart1, dart2 = darts1[i], darts2[i]
                    dart1.makeTwin(dart2)
                    edge = tiling.Edge(tiling, dart1)
                    dart1.edge = dart2.edge = edge

        # Now we need to deal with the outer face. 
        # First we need to split all of its edges appropriately to create subdarts. 
        # Then stitch them up with their twins. 
        for dart in oldOuterFace.darts():

            twinSubDarts = list(reversed(dart.twin.subdarts))
            dart.subdarts = [tiling.Dart(dcel = tiling, origin = twinSubDart.next.origin, face = tiling.outerFace)
                             for twinSubDart in twinSubDarts]

            for i in range(len(twinSubDarts)):
                twinSubDarts[i].makeTwin(dart.subdarts[i])
                edge = tiling.Edge(tiling, twinSubDarts[i])
                twinSubDarts[i].edge = dart.subdarts[i].edge = edge

            for i in range(len(dart.subdarts) - 1):
                dart.subdarts[i].makeNext(dart.subdarts[i+1])

        for dart in oldOuterFace.darts():
            dart.subdarts[-1].makeNext(dart.next.subdarts[0])

        tiling.outerFace.aDart = oldOuterFace.aDart.subdarts[0]

    for dart in tiling.darts:
        dart.splitVertices = []
        
    tiling.addLevel() # Push the current tiling level. 
    
    for tile in tiling.faceLevels[-1][1:]:
        subdivideTile(tiling, tile, tilingRules)
    
    stitchSubTilesTogether(tiling)
    
    for dart in tiling.darts:
        dart.origin.aDart = dart
        
def TilingViewer(packing, showCirclePacking = True, showTriangulation = True, size=(600,600)):
    import random

    from koebe.graphics.euclidean2viewer import PoincareDiskViewer, makeStyle
    from koebe.geometries.euclidean2 import PointE2, SegmentE2, PolygonE2

    def PointE2For(v):
        return PointE2(v.data.center.coord.real, v.data.center.coord.imag)

    def SegmentE2For(dart):
        return SegmentE2(PointE2For(dart.origin), PointE2For(dart.dest))

    def collectFaces(face):
        if len(face.subtiles) > 0:
            return [subface for subtile in face.subtiles for subface in collectFaces(subtile)]
        else: 
            return [face]
        
    viewer = PoincareDiskViewer(*size)
    
    # Uncomment to show super-tiles
    # level = 1
    # colors = ["#173679", "#0b1e38", "#db901c", "#e8e163", "#c6ca78", "#efe198", "hsl(120,100%,50%)"]
    # for i in range(1,len(original_pent_tiling.faceLevels[level])):
    #     rcolor = int(random.random() * 360)
    #     rsat =int(random.random()*50) + 50
    #     rlight = int(random.random()*40) + 50
    #     for tile in collectFaces(original_pent_tiling.faceLevels[level][i]):
    #         color = f"hsl({rcolor},{rsat}%,{rlight}%)"
    #         viewer.add(PolygonE2([PointE2For(v) for v in tile.vertices()]), 
    #                    makeStyle(fill=color))

    if showCirclePacking:
        circleStyle = makeStyle(stroke="#007849", strokeWeight=0.5)
        viewer.addAll([(v.data, circleStyle) for v in packing.verts])

    if showTriangulation:
        triSegStyle = makeStyle(stroke="#fece00", strokeWeight=1.0)
        triSegs = [(SegmentE2For(e.aDart), triSegStyle)
                   for e in packing.edges if not e.aDart.origin.is_tile_vertex or not e.aDart.dest.is_tile_vertex]
        viewer.addAll(triSegs)

    edgeStyle = makeStyle(stroke="#0375b4", strokeWeight=2.0)
    edgeSegs = [(SegmentE2For(e.aDart), edgeStyle)
               for e in packing.edges if e.aDart.origin.is_tile_vertex and e.aDart.dest.is_tile_vertex]
    viewer.addAll(edgeSegs)
    
    return viewer


def generateCirclePackingLayout(tiling, centerDartIdx = -1):
    # To circle pack we will have to triangulate each face, which adds
    # a new vertex for each face. We store the current vertex count
    # so we can distinguish between these new vertices and the originals
    # by index (index >= tile_vertex_count will be a triangulation
    # vertex)
    tile_vertex_count = len(tiling.verts)
    original_tiling = tiling
    duplicate_tiling = tiling.duplicate()
    starTriangulateAllFaces(duplicate_tiling)

    # Do the hyperbolic maximal circle packing
    from koebe.algorithms.hypPacker import maximalPacking
    packing, _ = maximalPacking(
        duplicate_tiling, 
        num_passes=1000,
        centerDartIdx=centerDartIdx
    )

    # Annotate each vertex with whether it is an original tile vertex
    # (i.e. .is_tile_vertex == True) or is one of the vertices added
    # to triangulate each face. 
    for vIdx in range(len(packing.verts)):
        packing.verts[vIdx].is_tile_vertex = vIdx < tile_vertex_count
    
    return packing, duplicate_tiling


# Chair Tiling

In [20]:
########################################
# Set up the finite subdivision rules
########################################

# The chair tiling only has one prototile
rules = TilingRules()
chair = rules.createPrototile("chair", ["A","B","C","D","E","F","G","H"])

# Edges that need to be split
chair.addSplitEdgeRules(((("A","B"), ("a")), (("B","C"), ("b")), (("C","D"), ("c")),
                        (("D","E"), ("d")), (("E","F"), ("e")), (("F","G"), ("f")),
                        (("G","H"), ("g")), (("H","A"), ("h"))))

# New vertices to create
chair.addNewVertexRules(("i","j","k","l","m"))

# The subdivision subtiles: 
chair.addSubtile("chair", ("A", "a", "B", "j", "k", "l", "H", "h"))
chair.addSubtile("chair", ("G", "g", "H", "l", "m", "e", "F", "f"))
chair.addSubtile("chair", ("k", "j", "i", "d", "E", "e", "m", "l"))
chair.addSubtile("chair", ("C", "c", "D", "d", "i", "j", "B", "b"))

########################################
# Apply the rules
########################################

tiling = rules.generateTiling("chair", depth = 6)

########################################
# Circle pack to get geometry. 
########################################

# To circle pack we will have to triangulate each face, which adds
# a new vertex for each face. We store the current vertex count
# so we can distinguish between these new vertices and the originals
# by index (index >= tile_vertex_count will be a triangulation
# vertex)
tile_vertex_count = len(tiling.verts)
print(tile_vertex_count)
#original_tiling = tiling
#tiling = tiling.duplicate()


12545


In [ ]:
starTriangulateAllFaces(tiling)

# Do the hyperbolic maximal circle packing
from koebe.algorithms.hypPacker import *
packing, _ = maximalPacking(
    tiling, 
    num_passes=1000
#      ,centerDartIdx=80
)

# Annotate each vertex with whether it is an original tile vertex
# (i.e. .is_tile_vertex == True) or is one of the vertices added
# to triangulate each face. 
for vIdx in range(len(packing.verts)):
    packing.verts[vIdx].is_tile_vertex = vIdx < tile_vertex_count

In [5]:
########################################
# Draw the tiling in the Poincare Disk
######################################## 

import random

from koebe.graphics.euclidean2viewer import PoincareDiskViewer, makeStyle
from koebe.geometries.euclidean2 import PointE2, SegmentE2, PolygonE2

def PointE2For(v):
    return PointE2(v.data.center.coord.real, v.data.center.coord.imag)

def SegmentE2For(dart):
    return SegmentE2(PointE2For(dart.origin), PointE2For(dart.dest))

viewer = PoincareDiskViewer(600, 600)

# Uncomment to show super-tiles
# level = 3
# colors = ["#173679", "#0b1e38", "#db901c", "#e8e163", "#c6ca78", "#efe198", "hsl(120,100%,50%)"]
# for i in range(1,len(original_tiling.faceLevels[level])):
#     rcolor = int(random.random() * 360)
#     rsat =int(random.random()*50) + 50
#     rlight = int(random.random()*40) + 50
#     for tile in collectFaces(original_tiling.faceLevels[level][i]):
#         color = f"hsl({rcolor},{rsat}%,{rlight}%)"
#         viewer.add(PolygonE2([PointE2For(v) for v in tile.vertices()]), 
#                    makeStyle(fill=color))

# Uncomment to view the circle packing: 
circleStyle = makeStyle(stroke="#007849", strokeWeight=0.5)
viewer.addAll([(v.data, circleStyle) for v in packing.verts])

# Ucomment to view the triangulation vertices
# triSegStyle = makeStyle(stroke="#fece00", strokeWeight=1.0)
# triSegs = [(SegmentE2For(e.aDart), triSegStyle)
#            for e in packing.edges if not e.aDart.origin.is_tile_vertex or not e.aDart.dest.is_tile_vertex]
# viewer.addAll(triSegs)

edgeStyle = makeStyle(stroke="#0375b4", strokeWeight=2.0)
edgeSegs = [(SegmentE2For(e.aDart), edgeStyle)
           for e in packing.edges if e.aDart.origin.is_tile_vertex and e.aDart.dest.is_tile_vertex]
viewer.addAll(edgeSegs)

def collectFaces(face):
    if len(face.subtiles) > 0:
        return [subface for subtile in face.subtiles for subface in collectFaces(subtile)]
    else: 
        return [face]

for vIdx in range(len(original_tiling.verts)):
    original_tiling.verts[vIdx].idx = vIdx
    original_tiling.verts[vIdx].data = packing.verts[vIdx].data

viewer.show()

<IPython.core.display.Javascript object>

E2Sketch(height=600, objects='[[{"type": "CircleE2", "center": [0, 0], "radius": 1.0, "style": {"stroke": "#00…

# Twisted pentagonal tiling

In [11]:
from koebe.algorithms.tiling import *
from koebe.algorithms.tutteEmbeddings import tutteEmbeddingE2

########################################
# Set up the finite subdivision rules
########################################

# The chair tiling only has one prototile
pent_rules = TilingRules()

pent = pent_rules.createPrototile("pent", tuple("ABCDE"))

# Edges that need to be split
pent.addSplitEdgeRules(((("A","B"), ("a", "b")), 
                        (("B","C"), ("c", "d")), 
                        (("C","D"), ("e", "f")),
                        (("D","E"), ("g", "h")), 
                        (("E","A"), ("i", "j"))))

# New vertices to create
pent.addNewVertexRules(("k"))

# The subdivision subtiles: 
pent.addSubtile("pent", tuple("Aabkj"))
pent.addSubtile("pent", tuple("Bcdkb"))
pent.addSubtile("pent", tuple("Cefkd"))
pent.addSubtile("pent", tuple("Dghkf"))
pent.addSubtile("pent", tuple("Eijkh"))

########################################
# Apply the rules
########################################

pent_tiling = pent_rules.generateTiling("pent", depth = 6)

########################################
# Circle pack to get geometry. 
########################################

# To circle pack we will have to triangulate each face, which adds
# a new vertex for each face. We store the current vertex count
# so we can distinguish between these new vertices and the originals
# by index (index >= tile_vertex_count will be a triangulation
# vertex)
pent_tile_vertex_count = len(pent_tiling.verts)
original_pent_tiling = pent_tiling
pent_tiling = pent_tiling.duplicate()
#starTriangulateAllFaces(pent_tiling)


In [19]:
starTriangulateAllFaces(pent_tiling)

# Do the hyperbolic maximal circle packing
from koebe.algorithms.hypPacker import *
pent_packing, _ = maximalPacking(
    pent_tiling, 
    num_passes=1000
#      ,centerDartIdx=80
)

# Annotate each vertex with whether it is an original tile vertex
# (i.e. .is_tile_vertex == True) or is one of the vertices added
# to triangulate each face. 
for vIdx in range(len(pent_packing.verts)):
    pent_packing.verts[vIdx].is_tile_vertex = vIdx < pent_tile_vertex_count

KeyboardInterrupt: 

In [14]:
########################################
# Draw the tiling in the Poincare Disk
######################################## 

import random

from koebe.graphics.euclidean2viewer import PoincareDiskViewer, makeStyle
from koebe.geometries.euclidean2 import PointE2, SegmentE2, PolygonE2

def PointE2For(v):
    return PointE2(v.data.center.coord.real, v.data.center.coord.imag)

def SegmentE2For(dart):
    return SegmentE2(PointE2For(dart.origin), PointE2For(dart.dest))

viewer = PoincareDiskViewer(600, 600)

# Uncomment to show super-tiles
# level = 1
# colors = ["#173679", "#0b1e38", "#db901c", "#e8e163", "#c6ca78", "#efe198", "hsl(120,100%,50%)"]
# for i in range(1,len(original_pent_tiling.faceLevels[level])):
#     rcolor = int(random.random() * 360)
#     rsat =int(random.random()*50) + 50
#     rlight = int(random.random()*40) + 50
#     for tile in collectFaces(original_pent_tiling.faceLevels[level][i]):
#         color = f"hsl({rcolor},{rsat}%,{rlight}%)"
#         viewer.add(PolygonE2([PointE2For(v) for v in tile.vertices()]), 
#                    makeStyle(fill=color))

# Uncomment to view the circle packing: 
circleStyle = makeStyle(stroke="#007849", strokeWeight=0.5)
viewer.addAll([(v.data, circleStyle) for v in pent_packing.verts])

# Ucomment to view the triangulation vertices
triSegStyle = makeStyle(stroke="#fece00", strokeWeight=1.0)
triSegs = [(SegmentE2For(e.aDart), triSegStyle)
           for e in pent_packing.edges if not e.aDart.origin.is_tile_vertex or not e.aDart.dest.is_tile_vertex]
viewer.addAll(triSegs)

edgeStyle = makeStyle(stroke="#0375b4", strokeWeight=2.0)
edgeSegs = [(SegmentE2For(e.aDart), edgeStyle)
           for e in pent_packing.edges if e.aDart.origin.is_tile_vertex and e.aDart.dest.is_tile_vertex]
viewer.addAll(edgeSegs)

def collectFaces(face):
    if len(face.subtiles) > 0:
        return [subface for subtile in face.subtiles for subface in collectFaces(subtile)]
    else: 
        return [face]

# for vIdx in range(len(original_pent_tiling.verts)):
#     original_tiling.verts[vIdx].idx = vIdx
#     original_tiling.verts[vIdx].data = packing.verts[vIdx].data

viewer.show()

<IPython.core.display.Javascript object>

E2Sketch(height=600, objects='[[{"type": "CircleE2", "center": [0, 0], "radius": 1.0, "style": {"stroke": "#00…

In [21]:
from koebe.algorithms.tutteEmbeddings import tutteEmbeddingE2
pent_tiling = tiling
tutteGraph = tutteEmbeddingE2(pent_tiling, False, True)

for vIdx in range(len(tutteGraph.verts)):
    tutteGraph.verts[vIdx].is_tile_vertex = vIdx < pent_tile_vertex_count

Duplicating DCEL...
done.
Reordering vertices...
done.
Laying out boundary...
done.
Computing graph laplacian...
Creating vertToIdx array...
Creating vertToDeg array...
Creating mat...
Done.
Returning coo_matrix with shape (12545, 12545)
done.
Computing Tutte embedding...
Computing P1...
Computing L1...
Computing B...
Computing L2...
Computing -inv(L2)
Computing nInvL2*B*P1
Concatenating P1 and P2
done.
Setting .data attributes...
done.


In [22]:
print(f"Vertex count: {len(tutteGraph.verts)}")
print(f"Vertex edges: {len(tutteGraph.edges)}")
print(f"Face count: {(len(tutteGraph.faces)-1)}")

Vertex count: 12545
Vertex edges: 16640
Face count: 4096


In [23]:
from koebe.geometries.euclidean2 import SegmentE2
from koebe.graphics.euclidean2viewer import UnitScaleE2Sketch, makeStyle
viewer = UnitScaleE2Sketch()

triSegStyle = makeStyle(stroke="#fece00", strokeWeight=0.5)
viewer.addAll(
    [(SegmentE2(e.aDart.origin.data, e.aDart.twin.origin.data), triSegStyle)
    for e in tutteGraph.edges if not e.aDart.origin.is_tile_vertex or not e.aDart.dest.is_tile_vertex]
)

edgeStyle = makeStyle(stroke="#0375b4", strokeWeight=0.5)
viewer.addAll(
    [(SegmentE2(e.aDart.origin.data, e.aDart.twin.origin.data), edgeStyle)
    for e in tutteGraph.edges if e.aDart.origin.is_tile_vertex and e.aDart.dest.is_tile_vertex]
)

viewer.show()

<IPython.core.display.Javascript object>

E2Sketch(height=500, objects='[[{"type": "SegmentE2", "endpoints": [[0.9999247018391445, 0.012271538285719925]…

# Twisted Hexagonal Tiling

In [602]:
########################################
# Set up the finite subdivision rules
########################################

# The chair tiling only has one prototile
hex_rules = TilingRules()

hex = hex_rules.createPrototile("hex", ["A_A", "B", "C", "D", "E", "F"])

# Edges that need to be split
hex.addSplitEdgeRules(( 
                        (("A_A","B"), ("a1", "a2", "a3")), 
                        (("B","C"), ("b1", "b2", "b3")), 
                        (("C","D"), ("c1", "c2", "c3")),
                        (("D","E"), ("d1", "d2", "d3")), 
                        (("E","F"), ("e1", "e2", "e3")),
                        (("F","A_A"), ("f1", "f2", "f3"))
                      ))

# New vertices to create
hex.addNewVertexRules(("x"))

# The subdivision subtiles: 
hex.addSubtile("hex", ("A_A", "a1", "a2", "a3", "x", "f3"))
hex.addSubtile("hex", ("B", "b1", "b2", "b3", "x", "a3"))
hex.addSubtile("hex", ("C", "c1", "c2", "c3", "x", "b3"))
hex.addSubtile("hex", ("D", "d1", "d2", "d3", "x", "c3"))
hex.addSubtile("hex", ("E", "e1", "e2", "e3", "x", "d3"))
hex.addSubtile("hex", ("F", "f1", "f2", "f3", "x", "e3"))

########################################
# Apply the rules
########################################

hex_tiling = hex_rules.generateTiling("hex", depth = 4)

########################################
# Circle pack to get geometry. 
########################################

# To circle pack we will have to triangulate each face, which adds
# a new vertex for each face. We store the current vertex count
# so we can distinguish between these new vertices and the originals
# by index (index >= tile_vertex_count will be a triangulation
# vertex)
hex_tile_vertex_count = len(hex_tiling.verts)
original_hex_tiling = hex_tiling
hex_tiling = hex_tiling.duplicate()
starTriangulateAllFaces(hex_tiling)

# Do the hyperbolic maximal circle packing
from koebe.algorithms.hypPacker import *
hex_packing, _ = maximalPacking(
    hex_tiling, 
    num_passes=1000
#      ,centerDartIdx=80
)

# Annotate each vertex with whether it is an original tile vertex
# (i.e. .is_tile_vertex == True) or is one of the vertices added
# to triangulate each face. 
for vIdx in range(len(hex_packing.verts)):
    hex_packing.verts[vIdx].is_tile_vertex = vIdx < hex_tile_vertex_count

In [603]:
########################################
# Draw the tiling in the Poincare Disk
######################################## 

import random

from koebe.graphics.euclidean2viewer import PoincareDiskViewer, makeStyle
from koebe.geometries.euclidean2 import PointE2, SegmentE2, PolygonE2

def PointE2For(v):
    return PointE2(v.data.center.coord.real, v.data.center.coord.imag)

def SegmentE2For(dart):
    return SegmentE2(PointE2For(dart.origin), PointE2For(dart.dest))

viewer = PoincareDiskViewer(600, 600)

# Uncomment to show super-tiles
# level = 1
# colors = ["#173679", "#0b1e38", "#db901c", "#e8e163", "#c6ca78", "#efe198", "hsl(120,100%,50%)"]
# for i in range(1,len(original_pent_tiling.faceLevels[level])):
#     rcolor = int(random.random() * 360)
#     rsat =int(random.random()*50) + 50
#     rlight = int(random.random()*40) + 50
#     for tile in collectFaces(original_pent_tiling.faceLevels[level][i]):
#         color = f"hsl({rcolor},{rsat}%,{rlight}%)"
#         viewer.add(PolygonE2([PointE2For(v) for v in tile.vertices()]), 
#                    makeStyle(fill=color))

# Uncomment to view the circle packing: 
circleStyle = makeStyle(stroke="#007849", strokeWeight=0.5)
viewer.addAll([(v.data, circleStyle) for v in hex_packing.verts])

# Ucomment to view the triangulation vertices
triSegStyle = makeStyle(stroke="#fece00", strokeWeight=1.0)
triSegs = [(SegmentE2For(e.aDart), triSegStyle)
           for e in hex_packing.edges if not e.aDart.origin.is_tile_vertex or not e.aDart.dest.is_tile_vertex]
viewer.addAll(triSegs)

edgeStyle = makeStyle(stroke="#0375b4", strokeWeight=2.0)
edgeSegs = [(SegmentE2For(e.aDart), edgeStyle)
           for e in hex_packing.edges if e.aDart.origin.is_tile_vertex and e.aDart.dest.is_tile_vertex]
viewer.addAll(edgeSegs)

def collectFaces(face):
    if len(face.subtiles) > 0:
        return [subface for subtile in face.subtiles for subface in collectFaces(subtile)]
    else: 
        return [face]

# for vIdx in range(len(original_pent_tiling.verts)):
#     original_tiling.verts[vIdx].idx = vIdx
#     original_tiling.verts[vIdx].data = packing.verts[vIdx].data

viewer.show()

<IPython.core.display.Javascript object>

E2Sketch(height=600, objects='[[{"type": "CircleE2", "center": [0, 0], "radius": 1.0, "style": {"stroke": "#00…

# Twisted n-agonal tiling

In [643]:
########################################
# Set up the finite subdivision rules
########################################

# The chair tiling only has one prototile
N = 9
ntile_rules = TilingRules()

ntile = ntile_rules.createPrototile("ntile", [f"V{i}" for i in range(N)])

# Edges that need to be split
ntile.addSplitEdgeRules([((f"V{i}",f"V{(i+1)%N}"), [f"v{i}_{j}" for j in range(1, N-2)])
                        for i in range(N)])

# New vertices to create
ntile.addNewVertexRules(("x"))

# The subdivision subtiles: 
for i in range(N):
    ntile.addSubtile("ntile", [f"V{i}"] + [f"v{i}_{j}" for j in range(1, N-2)] + ["x", f"v{(i+N-1)%N}_{N-3}"])

########################################
# Apply the rules
########################################

ntile_tiling = ntile_rules.generateTiling("ntile", depth = 4)

########################################
# Circle pack to get geometry. 
########################################

# To circle pack we will have to triangulate each face, which adds
# a new vertex for each face. We store the current vertex count
# so we can distinguish between these new vertices and the originals
# by index (index >= tile_vertex_count will be a triangulation
# vertex)
ntile_tile_vertex_count = len(ntile_tiling.verts)
original_ntile_tiling = ntile_tiling
ntile_tiling = ntile_tiling.duplicate()
starTriangulateAllFaces(ntile_tiling)

# Do the hyperbolic maximal circle packing
from koebe.algorithms.hypPacker import *
ntile_packing, _ = maximalPacking(
    ntile_tiling, 
    num_passes=1000
#      ,centerDartIdx=80
)

# Annotate each vertex with whether it is an original tile vertex
# (i.e. .is_tile_vertex == True) or is one of the vertices added
# to triangulate each face. 
for vIdx in range(len(ntile_packing.verts)):
    ntile_packing.verts[vIdx].is_tile_vertex = vIdx < ntile_tile_vertex_count

In [642]:
########################################
# Draw the tiling in the Poincare Disk
######################################## 

import random

from koebe.graphics.euclidean2viewer import PoincareDiskViewer, makeStyle
from koebe.geometries.euclidean2 import PointE2, SegmentE2, PolygonE2

def PointE2For(v):
    return PointE2(v.data.center.coord.real, v.data.center.coord.imag)

def SegmentE2For(dart):
    return SegmentE2(PointE2For(dart.origin), PointE2For(dart.dest))

viewer = PoincareDiskViewer(600, 600)

# Uncomment to show super-tiles
# level = 1
# colors = ["#173679", "#0b1e38", "#db901c", "#e8e163", "#c6ca78", "#efe198", "hsl(120,100%,50%)"]
# for i in range(1,len(original_pent_tiling.faceLevels[level])):
#     rcolor = int(random.random() * 360)
#     rsat =int(random.random()*50) + 50
#     rlight = int(random.random()*40) + 50
#     for tile in collectFaces(original_pent_tiling.faceLevels[level][i]):
#         color = f"hsl({rcolor},{rsat}%,{rlight}%)"
#         viewer.add(PolygonE2([PointE2For(v) for v in tile.vertices()]), 
#                    makeStyle(fill=color))

# Uncomment to view the circle packing: 
circleStyle = makeStyle(stroke="#007849", strokeWeight=0.5)
viewer.addAll([(v.data, circleStyle) for v in ntile_packing.verts])

# Ucomment to view the triangulation vertices
triSegStyle = makeStyle(stroke="#fece00", strokeWeight=1.0)
triSegs = [(SegmentE2For(e.aDart), triSegStyle)
           for e in ntile_packing.edges if not e.aDart.origin.is_tile_vertex or not e.aDart.dest.is_tile_vertex]
viewer.addAll(triSegs)

edgeStyle = makeStyle(stroke="#0375b4", strokeWeight=2.0)
edgeSegs = [(SegmentE2For(e.aDart), edgeStyle)
           for e in ntile_packing.edges if e.aDart.origin.is_tile_vertex and e.aDart.dest.is_tile_vertex]
viewer.addAll(edgeSegs)

def collectFaces(face):
    if len(face.subtiles) > 0:
        return [subface for subtile in face.subtiles for subface in collectFaces(subtile)]
    else: 
        return [face]

# for vIdx in range(len(original_pent_tiling.verts)):
#     original_tiling.verts[vIdx].idx = vIdx
#     original_tiling.verts[vIdx].data = packing.verts[vIdx].data

viewer.show()

<IPython.core.display.Javascript object>

E2Sketch(height=600, objects='[[{"type": "CircleE2", "center": [0, 0], "radius": 1.0, "style": {"stroke": "#00…

# Two-Prototile Experimental

In [10]:
from koebe.algorithms.tiling import *
from koebe.algorithms.tutteEmbeddings import tutteEmbeddingE2

########################################
# Set up the finite subdivision rules
########################################

# The chair tiling only has one prototile
rules = TilingRules()

t0 = rules.createPrototile("t0", tuple("AB"))
t1 = rules.createPrototile("t1", tuple("ABC"))
t2 = rules.createPrototile("t2", tuple("ABCDEF"))
t3 = rules.createPrototile("t3", tuple("ABCD"))

# Edges that need to be split
t1.addSplitEdgeRules((
    (("A", "B"), ("a1", "a2")),
    (("B", "C"), ("b")),
    (("C", "A"), ("c"))
))
t2.addSplitEdgeRules((
    (("A", "B"), ("a1", "a2")),
    (("B", "C"), ("b1", "b2")),
    (("C", "D"), ("c1", "c2")),
    (("D", "E"), ["d1"]),
    (("E", "F"), ("e1", "e2")),
    (("F", "A"), ["f1"]),
))
t3.addSplitEdgeRules((
    (("A", "B"), ["a1", "a2"]), 
    (("B", "C"), ["b1"]), 
    (("C", "D"), ["c1", "c2"]), 
    (("D", "A"), ["d1"])
))

# New vertices to create
t0.addNewVertexRules(("x"))
t2.addNewVertexRules(("x", "y"))
t3.addNewVertexRules(("x", "y"))

# The subdivision subtiles: 
t0.addSubtile("t1", ("A", "B", "x"))
t0.addSubtile("t1", ("B", "A", "x"))
#t0.addSubtile("t1", ("C", "A", "x"))

t1.addSubtile("t1", ("c", "b", "C"))
t1.addSubtile("t2", ("A", "a1", "a2", "B", "b", "c"))

t2.addSubtile("t3", ("f1", "x", "e2", "F"))
t2.addSubtile("t3", ("x", "y", "e1", "e2"))
t2.addSubtile("t3", ("y", "d1", "E", "e1"))
t2.addSubtile("t2", ("A", "a1", "a2", "B", "x", "f1"))
t2.addSubtile("t2", ("B", "b1", "b2", "C", "y", "x"))
t2.addSubtile("t2", ("C", "c1", "c2", "D", "d1", "y"))

t3.addSubtile("t3", ("A", "a1", "x", "d1"))
t3.addSubtile("t3", ("a1", "a2", "y", "x"))
t3.addSubtile("t3", ("a2", "B", "b1", "y"))
t3.addSubtile("t3", ("y", "b1", "C", "c1"))
t3.addSubtile("t3", ("x", "y", "c1", "c2"))
t3.addSubtile("t3", ("d1", "x", "c2", "D"))


########################################
# Apply the rules
########################################

print("Generating tiling...")
tiling = rules.generateTiling("t0", depth = 8)
print(len(tiling.verts))
print(len(tiling.darts))
print(len(tiling.faces))
tile_vertex_count = len(tiling.verts)

########################################
# Circle pack to get geometry. 
########################################

# print("Circle packing...")
# import koebe.algorithms.hypPacker
# koebe.algorithms.hypPacker._REPACK_DEBUG = True
# packing, _ = generateCirclePackingLayout(tiling)

Generating tiling...
116349
456648
111977


In [5]:
from koebe.algorithms.tutteEmbeddings import tutteEmbeddingE2
tutteGraph = tutteEmbeddingE2(tiling, False, True)

for vIdx in range(len(tutteGraph.verts)):
    tutteGraph.verts[vIdx].is_tile_vertex = vIdx < tile_vertex_count

Duplicating DCEL...
done.
Reordering vertices...
done.
Laying out boundary...
done.
Computing graph laplacian...
Creating vertToIdx array...
Creating vertToDeg array...
Creating mat...
Done.
Returning coo_matrix with shape (20121, 20121)
done.
Computing Tutte embedding...
Computing P1...
Computing L1...
Computing B...
Computing L2...
Computing -inv(L2)
Computing nInvL2*B*P1
Concatenating P1 and P2
done.
Setting .data attributes...
done.


NameError: name 'tile_vertex_count' is not defined

In [9]:
tile_vertex_count = 20121
for vIdx in range(len(tutteGraph.verts)):
    tutteGraph.verts[vIdx].is_tile_vertex = vIdx < tile_vertex_count

from koebe.geometries.euclidean2 import SegmentE2
from koebe.graphics.euclidean2viewer import UnitScaleE2Sketch, makeStyle
viewer = UnitScaleE2Sketch()

triSegStyle = makeStyle(stroke="#fece00", strokeWeight=0.5)
viewer.addAll(
    [(SegmentE2(e.aDart.origin.data, e.aDart.twin.origin.data), triSegStyle)
    for e in tutteGraph.edges if not e.aDart.origin.is_tile_vertex or not e.aDart.dest.is_tile_vertex]
)

edgeStyle = makeStyle(stroke="#0375b4", strokeWeight=0.5)
viewer.addAll(
    [(SegmentE2(e.aDart.origin.data, e.aDart.twin.origin.data), edgeStyle)
    for e in tutteGraph.edges if e.aDart.origin.is_tile_vertex and e.aDart.dest.is_tile_vertex]
)

viewer.show()

<IPython.core.display.Javascript object>

E2Sketch(height=500, objects='[[{"type": "SegmentE2", "endpoints": [[0.0009220693342046941, -3.973640849762822…

In [12]:
########################################
# Draw the tiling in the Poincare Disk
######################################## 

TilingViewer(packing).show()

<IPython.core.display.Javascript object>

E2Sketch(height=600, objects='[[{"type": "CircleE2", "center": [0, 0], "radius": 1.0, "style": {"stroke": "#00…

# Some older stuff that may be useful for debugging so I'm not deleting it yet. 

In [499]:
########################################
# Draw the tiling in the Poincare Disk
######################################## 

from koebe.graphics.euclidean2viewer import PoincareDiskViewer, makeStyle
from koebe.geometries.euclidean2 import PointE2, SegmentE2, PolygonE2

def PointE2For(v):
    return PointE2(v.data.center.coord.real, v.data.center.coord.imag)

def SegmentE2For(dart):
    return SegmentE2(PointE2For(dart.origin), PointE2For(dart.dest))

viewer = PoincareDiskViewer(400, 400)

# Uncomment to show super-tiles
level = 3
# colors = ["#173679", "#0b1e38", "#db901c", "#e8e163"]
# for i in range(1,len(original_tiling.faceLevels[level])):
#     for tile in collectFaces(original_tiling.faceLevels[level][i]):
#         viewer.add(PolygonE2([PointE2For(v) for v in tile.vertices()]), 
#                    makeStyle(fill=colors[i%4]))

# Uncomment to view the circle packing: 
circleStyle = makeStyle(stroke="#007849", strokeWeight=0.5)
viewer.addAll([(v.data, circleStyle) for v in packing.verts])

# Ucomment to view the triangulation vertices
triSegStyle = makeStyle(stroke="#fece00", strokeWeight=1.0)
triSegs = [(SegmentE2For(e.aDart), triSegStyle)
           for e in packing.edges if not e.aDart.origin.is_tile_vertex or not e.aDart.dest.is_tile_vertex]
viewer.addAll(triSegs)

edgeStyle = makeStyle(stroke="#0375b4", strokeWeight=2.0)
edgeSegs = [(SegmentE2For(e.aDart), edgeStyle)
           for e in packing.edges if e.aDart.origin.is_tile_vertex and e.aDart.dest.is_tile_vertex]
viewer.addAll(edgeSegs)

def collectFaces(face):
    if len(face.subtiles) > 0:
        return [subface for subtile in face.subtiles for subface in collectFaces(subtile)]
    else: 
        return [face]

for vIdx in range(len(original_tiling.verts)):
    original_tiling.verts[vIdx].idx = vIdx
    original_tiling.verts[vIdx].data = packing.verts[vIdx].data

viewer.show()

<IPython.core.display.Javascript object>

E2Sketch(objects='[[{"type": "CircleE2", "center": [0, 0], "radius": 1.0, "style": {"stroke": "#000", "strokeW…

In [430]:
# Useful debugging checks: 

tile_vertex_count = len(tiling.verts)

dartEdgeError = False
dartTwinError = False
dartNextPrevError = False
dartFaceError = False
dartVertexError = False

for dart in tiling.darts:
    if dart != dart.edge.aDart and dart.twin != dart.edge.aDart:
        dartEdgeError = True
    if dart.twin.twin != dart:
        dartTwinError = True
    if dart.next.prev != dart:
        dartNextPrevError = True
    if dart not in dart.face.darts():
        dartFaceError = True
    if dart not in dart.origin.outDarts():
        dartVertexError = True

vertexDartError = False
for vertex in tiling.verts:
    for dart in vertex.outDarts():
        if vertex != dart.origin:
            print(tiling.verts.index(vertex))
            vertexDartError = True
            
print(f"Detected a dartEdgeError: {dartEdgeError}")
print(f"Detected a dartTwinError: {dartTwinError}")
print(f"Detected a dartNextPrevError: {dartNextPrevError}")
print(f"Detected a dartFaceError: {dartFaceError}")
print(f"Detected a dartVertexError: {dartVertexError}")
print(f"Detected a vertexDartError: {vertexDartError}")

Detected a dartEdgeError: False
Detected a dartTwinError: False
Detected a dartNextPrevError: False
Detected a dartFaceError: False
Detected a dartVertexError: False
Detected a vertexDartError: False


In [438]:
from koebe.algorithms.hypPacker import *
packing, _ = maximalPacking(
    tiling, 
    num_passes=1000
)

for vIdx in range(len(packing.verts)):
    packing.verts[vIdx].is_tile_vertex = vIdx < tile_vertex_count

In [439]:
from koebe.graphics.euclidean2viewer import PoincareDiskViewer, makeStyle
from koebe.geometries.euclidean2 import PointE2, SegmentE2

def PointE2For(v):
    return PointE2(v.data.center.coord.real, v.data.center.coord.imag)

def SegmentE2For(dart):
    return SegmentE2(PointE2For(dart.origin), PointE2For(dart.dest))

viewer = PoincareDiskViewer(400, 400)
#viewer.addAll([v.data for v in packing.verts])
edgeSegs = [SegmentE2For(e.aDart)
           for e in packing.edges if e.aDart.origin.is_tile_vertex and e.aDart.dest.is_tile_vertex]
viewer.addAll(edgeSegs)

def showDartAndNext(viewer, dart):
    viewer.add(SegmentE2For(dart),      makeStyle(stroke="#0f0", strokeWeight=3.0))
    viewer.add(SegmentE2For(dart.next), makeStyle(stroke="#f00", strokeWeight=3.0))

def showDartAndTwin(viewer, dart):
    viewer.add(SegmentE2For(dart),      makeStyle(stroke="#0f0", strokeWeight=6.0))
    viewer.add(SegmentE2For(dart.twin), makeStyle(stroke="#f00", strokeWeight=3.0))

def showOutDarts(viewer, vertex):
    for dart in vertex.outDarts(): viewer.add(SegmentE2For(dart), makeStyle(stroke="#f00", strokeWeight=6.0))
    viewer.add(PointE2For(vertex), makeStyle(fill="#0f0"))
    
def showInDarts(viewer, vertex):
    for dart in vertex.inDarts(): viewer.add(SegmentE2For(dart), makeStyle(stroke="#f00", strokeWeight=6.0))
    viewer.add(PointE2For(vertex), makeStyle(fill="#0f0"))

# showDartAndTwin(viewer, packing.darts[82])
# showDartAndTwin(viewer, packing.darts[85])

#i += 1
#showInDarts(viewer, packing.verts[i])
viewer.show()

<IPython.core.display.Javascript object>

E2Sketch(objects='[[{"type": "CircleE2", "center": [0, 0], "radius": 1.0, "style": {"stroke": "#000", "strokeW…

In [160]:
packing.verts[0].data

CircleH2(center=PointH2(coord=ExtendedComplex(z=(-0.11851176705023778-0.9929526479498557j), w=(1+0j), _complex=(-0.11851176705023778-0.9929526479498557j))), xRadius=-0.07986492414663876)

In [7]:
from koebe.geometries.euclidean2 import PointE2, SegmentE2
viewer = PoincareDiskViewer(600, 600)
edgeSegs = [SegmentE2(PointE2(e.aDart.origin.data.center.coord.real, e.aDart.origin.data.center.coord.imag), 
                      PointE2(e.aDart.dest.data.center.coord.real, e.aDart.dest.data.center.coord.imag))
           for e in packing.edges]
viewer.addAll(edgeSegs)
blue = makeStyle(stroke = "#79f", strokeWeight=1.0)
red = makeStyle(stroke = "#f97", strokeWeight=1.0)
green = makeStyle(stroke = "#7f9", strokeWeight=1.0)
gray = makeStyle(stroke = "#999", strokeWeight=1.0)
for i in range(len(edgeSegs)):
    e = edgeSegs[i]
    if tiling.edges[i].level % 4 == 0:
        viewer.setStyle(e, blue)
    elif tiling.edges[i].level % 4 == 1:
        viewer.setStyle(e, red)
    elif tiling.edges[i].level % 4 == 2:
        viewer.setStyle(e, green)
    else:
        viewer.setStyle(e, gray)
viewer.show()

<IPython.core.display.Javascript object>

AttributeError: 'Edge' object has no attribute 'level'

In [60]:
tiling.__class__

__main__.Tiling

In [5]:
tiling = Tiling.generateCycle(vdata = ["A","B","C","D","E","F","G","H"])

In [7]:
tiling.darts

In [451]:
class Foo:
    pass

x = Foo()

x.myAttribute = "now it does"

In [452]:
x.myAttribute

'now it does'